In [16]:

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import praw
import os
from pathlib import Path
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from dotenv import load_dotenv


In [21]:
load_dotenv()
client_id = os.getenv("reddit_client_id")
client_secret = os.getenv("reddit_client_secret")
user_agent = os.getenv("reddit_user_agent")
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent,
    username ="No_Appearance_1995",
    password="Gmail@123"  # Replace with your Reddit password

)
reddit.subreddit("test").submit("Test Submission", url="https://reddit.com")

Submission(id='1lyw4xw')